In [101]:
from PIL import Image
from time import time_ns
import math
import numpy as np

In [102]:
def getImages():
    image_1 = Image.open('input/84_3.png', 'r').convert('RGB')
    image_2 = Image.open('input/84_95.png', 'r').convert('RGB')
    image_3 = Image.open('input/198_115.png', 'r').convert('RGB')
    image_4 = Image.open('input/198_153.png', 'r').convert('RGB')
    image_5 = Image.open('input/198_267.png', 'r').convert('RGB')
    image_6 = Image.open('input/207_12.png', 'r').convert('RGB')
    image_7 = Image.open('input/im1.png', 'r').convert('RGB')
    image_8 = Image.open('input/im2.png', 'r').convert('RGB')
    image_9 = Image.open('input/im3.png', 'r').convert('RGB')
    return {'first': image_1, 'second': image_2, 'third': image_3, 'fourth': image_4, 'fifth': image_5, 'sixth': image_6, 'seventh': image_7, 'eigth': image_8, 'ninth': image_9}


In [103]:
def printAudit(test_name, delta_time, audit):   #Просто получение инфы для тестов
  print(f'\nTest {test_name} passed: {(delta_time) / math.pow(10, 9)}s')
  for i in audit:
    name = i.get('name')
    input_size = i.get('input_size')
    output_size = i.get('output_size')
    print(f'Image {name}; input_size: {input_size}; output_size: {output_size}')

In [104]:
#def pixel_mean_value_grayscale(pixel) -> int:
#  return int(round((pixel[0] + pixel[1] + pixel[2]) / 3))

In [105]:
def semitone(image) -> Image:
  result = Image.new('L', (image.width, image.height))
  for x in range(result.width):
    for y in range(result.height):
      pixel = image.getpixel((x, y))
      new_pixel = int(round((pixel[0] + pixel[1] + pixel[2]) / 3))  #Рассчитываем одну матрицу Y на основе исходного Y = (R+G+B)/3
      result.putpixel((x, y), new_pixel)
  return result

In [106]:
def semitone_test():
  images = getImages()

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    newImage = semitone(image)
    output_name = f'output/1/{image_name}.processed.png'
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
    })
    newImage.save(output_name)

  end_time = time_ns()
  printAudit('semitone', end_time - start_time, audit)

In [107]:
def singh_adaptive_threshold(image, k=0.06, window_size=15):  #мб протестирую k=0.03 & size=25, а то 40 минут вообще дерьмово
        assert 0 <= k <= 1
        assert window_size % 2 == 1
        img_array = np.array(image, dtype=np.float32)
        binary_image = np.zeros_like(img_array, dtype=np.uint8)
        half_window = window_size // 2  #для определения локального окна вокруг каждого пикселя
        padded_img = np.pad(img_array, ((half_window, half_window), (half_window, half_window)), 'reflect') #дополняем изображение отражением пикселей изображения, чтобы обеспечить локальные окна одинакового размера для крайних пикселей
        for y in range(half_window, padded_img.shape[0] - half_window):
                for x in range(half_window, padded_img.shape[1] - half_window): #извлекаем локальное окно из дополненного изображения. Выбирается квадратная область, центром которой является текущий пиксель
                        window = padded_img[y - half_window:y + half_window + 1, x - half_window:x + half_window + 1]
                        local_mean = np.mean(window)
                        local_std = np.std(window)
                        threshold = local_mean * (1 + k * ((local_std / (1 - local_std)) - 1))  #пороговое значение
                        if img_array[y - half_window, x - half_window] > threshold:
                                binary_image[y - half_window, x - half_window] = 255  #белый, если больше порогового значения
                        else:
                                binary_image[y - half_window, x - half_window] = 0 #black 
        return Image.fromarray(binary_image)

In [108]:
def getSemitoneImages():
    image_1 = Image.open('output/1/first.processed.png', 'r')
    image_2 = Image.open('output/1/second.processed.png', 'r')
    image_3 = Image.open('output/1/third.processed.png', 'r')
    image_4 = Image.open('output/1/fourth.processed.png', 'r')
    image_5 = Image.open('output/1/fifth.processed.png', 'r')
    image_6 = Image.open('output/1/sixth.processed.png', 'r')
    image_7 = Image.open('output/1/seventh.processed.png', 'r')
    image_8 = Image.open('output/1/eigth.processed.png', 'r')
    image_9 = Image.open('output/1/ninth.processed.png', 'r')
    return {'first': image_1, 'second': image_2, 'third': image_3, 'fourth': image_4, 'fifth': image_5, 'sixth': image_6, 'seventh': image_7, 'eigth': image_8, 'ninth': image_9}

In [109]:
def singh_adaptive_threshold_test():
    images = getSemitoneImages()

    start_time = time_ns()
    audit = []
    for (image_name, image) in images.items():
        #check_image = semitone(image)  # Можно с проверкой, можно и нет
        newImage = singh_adaptive_threshold(image)
        output_name = f'output/2.9/{image_name}.processed.png'
        audit.append({
            'name': output_name,
            'input_size': f'{image.size[0]}x{image.size[1]}',
            'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
        })
        newImage.save(output_name)

    end_time = time_ns()
    printAudit('adaptive_threshold', end_time - start_time, audit)

In [110]:
def main():
    #semitone_test()
    singh_adaptive_threshold_test()

In [111]:
if __name__ == '__main__':
  main()


Test adaptive_threshold passed: 2529.9035819s
Image output/2.9/first.processed.png; input_size: 3639x2381; output_size: 3639x2381
Image output/2.9/second.processed.png; input_size: 1280x813; output_size: 1280x813
Image output/2.9/third.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/2.9/fourth.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/2.9/fifth.processed.png; input_size: 3050x4050; output_size: 3050x4050
Image output/2.9/sixth.processed.png; input_size: 3000x4250; output_size: 3000x4250
Image output/2.9/seventh.processed.png; input_size: 1694x929; output_size: 1694x929
Image output/2.9/eigth.processed.png; input_size: 1408x812; output_size: 1408x812
Image output/2.9/ninth.processed.png; input_size: 1413x814; output_size: 1413x814
